In [3]:
# import kagglehub
# import os

# data_dir = "../.cache"
# os.makedirs(data_dir, exist_ok=True)

# os.environ['KAGGLE_CACHE_DIR'] = data_dir

# path = kagglehub.dataset_download("mrdaniilak/russia-real-estate-20182021")

# print("Path to dataset files:", path)

In [6]:
import pandas as pd 
df = pd.read_csv('../.cache/all_v2.csv')

In [7]:
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt

sochi_polygon = Polygon([
    (39.0, 43.0),
    (40.5, 43.0),
    (40.5, 44.0),
    (39.0, 44.0)
])

def is_in_sochi(row):
    point = Point(row['geo_lon'], row['geo_lat'])
    return sochi_polygon.contains(point)

df['in_sochi'] = df.apply(is_in_sochi, axis=1)
sochi_points = df[df['in_sochi']].drop(columns=['in_sochi'])

sochi_points.to_csv('../data/sochi_real_estate_full.csv', index=False)

print(f"Filtered dataset saved with {len(sochi_points)} points.")

Filtered dataset saved with 67529 points.


In [9]:
import pandas as pd
import folium
from folium.plugins import HeatMap

sochi_points['year'] = pd.to_datetime(sochi_points['date']).dt.year

m = folium.Map(location=[43.6, 39.7], zoom_start=10)

for year in sochi_points['year'].unique():
    year_data = sochi_points[sochi_points['year'] == year]
    heat_data = [[row['geo_lat'], row['geo_lon']] for _, row in year_data.iterrows()]
    HeatMap(heat_data, name=f'Year {year}').add_to(m)

folium.LayerControl().add_to(m)
m.save('sochi_heatmap.html')
print("Interactive map saved as 'sochi_heatmap.html'.")

Interactive map saved as 'sochi_heatmap.html'.
